In [1]:
import torchvision.transforms as transforms
import torchvision.datasets as datasets
import torch
import torchvision.models as models
import torch.optim as optimizer
import torch.nn.functional as F

In [2]:
DATASET = 'dataset-more'
PATH = 'sll-more.pth'

In [3]:
DATASET = 'dataset-less'
PATH = 'sll-less.pth'

In [4]:
TRANSFORMS = transforms.Compose([
    transforms.ColorJitter(0.2, 0.2, 0.2, 0.2),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

BATCH_SIZE = 8

#create dataset from folder
dataset = datasets.ImageFolder(
    DATASET,
    TRANSFORMS
)
print('dataset created')

#create loader
train_loader = torch.utils.data.DataLoader(
    dataset,
    batch_size = BATCH_SIZE,
    shuffle = True
)
print('loader created')

#use pretrained Alexnet as model
model = models.alexnet(pretrained=True)
print('model init')

#change last layer to number of categories (27 for 26 letters + empty background)
model.classifier[6] = torch.nn.Linear(model.classifier[6].in_features, 27)

#cuda
device = torch.device('cuda')
model = model.to(device)
print('all set')

dataset created
loader created
model init
all set


In [ ]:
%%time
#train
EPOCHS = 10
BEST_ACC = 0

print('training start, freezing may occur')
optimize = optimizer.SGD(model.parameters(), lr=1e-3, momentum=0.9)
model.train()
for epoch in range (EPOCHS):
    i = 0
    sum_loss = 0.0
    error_count = 0.0
    for images, labels in iter(train_loader):
        # send data to device
        images = images.to(device)
        labels = labels.to(device)

        # zero gradients of parameters
        optimize.zero_grad()

        # execute model to get outputs
        outputs = model(images)

        # compute loss
        loss = F.cross_entropy(outputs, labels)

        # run backpropogation to accumulate gradients
        loss.backward()

        # step optimizer to adjust parameters
        optimize.step()

        # increment progress
        error_count += len(torch.nonzero(outputs.argmax(1) - labels).flatten())
        count = len(labels.flatten())
        i += count
        sum_loss += float(loss)
        
    accuracy = 1.0 - error_count / i
    print('epoch %d: accuracy %f, loss %f' % (epoch, accuracy,sum_loss / i))
    if accuracy > BEST_ACC:
        BEST_ACC = accuracy
        torch.save(model.state_dict(), PATH)

training start, freezing may occur


In [ ]:
import os
os._exit(00)